[View in Colaboratory](https://colab.research.google.com/github/farahbakhsh3/X1/blob/master/Ferdosi4.ipynb)

In [0]:
# !wget https://www.dropbox.com/s/laax0sxqzzlddtq/ferdosi.txt
# !wget https://www.dropbox.com/s/ripsc7hfc0nihy5/test.txt

In [0]:
import random
 
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout
from keras.layers import LSTM, GRU, CuDNNGRU, CuDNNLSTM 
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence

from keras.losses import categorical_crossentropy
from keras.utils import to_categorical

import tensorflow as tf

In [124]:
path = './ferdosi.txt'
with open(path, encoding='utf-8') as f:
    texts = f.read()

beits = texts.split('\n')
# texts = texts.split('\n')
texts = texts.replace('\n', 'ــ ')
texts = texts.split(' ')
# texts = text_to_word_sequence(texts)

print('corpus length:', len(texts))

corpus length: 561804


In [125]:
max_length = 8
embeding_dim = 100
step = 5


tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
sequences = [sequences[i][0] for i in range(len(sequences)-1)]

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

len_of_words = len(word_index)
word2idx = word_index
idx2word = dict([(value, key) for (key, value) in word_index.items()])

# paded_sequences = pad_sequences(sequences, maxlen=max_length)
# paded_sequences = [paded_sequences[i][0] for i in range(len(paded_sequences)-1)]

Found 21963 unique tokens.


In [126]:

sentences = []
next_word = []
for i in range(0, len(sequences) - max_length, step):
    sentences.append(sequences[i: i + max_length])
    next_word.append(sequences[i + max_length])
print('nb sentences:', len(sentences))
print('nb next_word:', len(next_word))

nb sentences: 112359
nb next_word: 112359


In [127]:
for ii in range(3):

    print(sentences[ii])
    print([idx2word.get(i, '') for i in sentences[ii]])

    print(next_word[ii])
    print(idx2word[next_word[ii]])


[2, 95, 339, 103, 1, 368, 344, 705]
['به', 'نام', 'خداوند', 'جان', 'و', 'خردــ', 'کزین', 'برتر']
215
اندیشه
[368, 344, 705, 215, 6225, 339, 95, 1]
['خردــ', 'کزین', 'برتر', 'اندیشه', 'برنگذردــ', 'خداوند', 'نام', 'و']
339
خداوند
[339, 95, 1, 339, 182, 339, 685, 282]
['خداوند', 'نام', 'و', 'خداوند', 'جایــ', 'خداوند', 'روزی', 'ده']
488
رهنمایــ


In [0]:
x = np.array(sentences)
y = np.array(next_word)
# y = to_categorical(y)

In [0]:
def on_epoch_end(epoch, _):
#     if epoch % 10 != 0:
#         return
    
    start_index = random.randint(0, len(beits) - 1)
    for diversity in [1.0]:
        print('----- diversity:', diversity)

        sentence = beits[start_index] + 'ــ'
        print(sentence)
        generated = sentence
        print('----- Generating with seed: "' + sentence + '"')
        for i in range(max_length * 10):
            x_pred = np.zeros((1, max_length), dtype=int)
            for t, word in enumerate(sentence.split(' ')[:max_length]):
                try:
#                     print(t, word)
                    x_pred[0, t] = word2idx[word]
                except:
                    pass

            preds = model.predict(x_pred, verbose=0)[0]
            pred_id = sample(preds, diversity)
            next_word = idx2word[pred_id]

            generated = generated + ' ' + next_word
            sentence = ' '.join(sentence.split(' ')[1:])
            sentence = sentence + ' ' + next_word

        print(generated.replace('ــ', '\n'))

In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [131]:
# model = models.Sequential()
# model.add(layers.Embedding(len(word_index), 100))
# model.add(layers.LSTM(128))
# model.add(layers.Dense(1, activation='sigmoid'))

# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])



model = Sequential()
model.add(Embedding(len_of_words, embeding_dim, input_length=max_length))
if tf.test.is_gpu_available():
#     model.add(CuDNNGRU(512, return_sequences=True))
    model.add(CuDNNGRU(256))
else:
    model.add(GRU(256))
model.add(Dense(len_of_words, activation='softmax'))
 
model.compile(optimizer=RMSprop(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
 
print(model.summary())
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 8, 100)            2196300   
_________________________________________________________________
cu_dnngru_14 (CuDNNGRU)      (None, 256)               274944    
_________________________________________________________________
dense_13 (Dense)             (None, 21963)             5644491   
Total params: 8,115,735
Trainable params: 8,115,735
Non-trainable params: 0
_________________________________________________________________
None


In [0]:

history = model.fit(x=x, y=y, epochs=150, batch_size=128, callbacks=[print_callback], validation_split=0.1)


Train on 101123 samples, validate on 11236 samples
Epoch 1/150
101123/101123 [==============================] - 22s 218us/step - loss: 7.1685 - acc: 0.0804 - val_loss: nan - val_acc: 0.0933
----- diversity: 1.0
دگر هرچ بودش به درویش دادــ
----- Generating with seed: "دگر هرچ بودش به درویش دادــ"
دگر هرچ بودش به درویش داد
 بدو اندر گردان به به نیز
 سواران و آمدند
 بران و یل همانا نخوانمت و سوی بباید اکنون که اندر و بزد بد خویشتن نازی و و همه چنین برآشفت و تا تخت
 سخنها و رخ و که به و که کار
 پدر
 به تارک کن بد چشم
 چون یکی ره بیامد و سرود
 بر روشنروان
 همیشه بر داند اندرین ببند
 به و کین
 سپاه سر و گرگ و از ز نیاید یکی دژ شاه همه و چنان و چنین
Epoch 2/150
101123/101123 [==============================] - 21s 206us/step - loss: 6.6962 - acc: 0.1195 - val_loss: nan - val_acc: 0.1000
----- diversity: 1.0
وگر تیغ زن نامداری سترگــ
----- Generating with seed: "وگر تیغ زن نامداری سترگــ"
وگر تیغ زن نامداری سترگ
 بود
 بود
 بود
 شتاب
 بدو که برین به گودرز زان شد آفرین چنین مردم زبان کز به همی از

In [0]:
import matplotlib.pyplot as plt

In [0]:
history_dict = history.history

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

epochs = range(1, len(loss_values) + 1)

plt.plot(epochs, loss_values, 'bo', label='Training Loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation Loss')
plt.plot(epochs, acc_values, 'ro', label='Training acc')
plt.plot(epochs, val_acc_values, 'r', label='Validation acc')
plt.title('Training & Validation Loss - acc')
plt.xlabel('Epochs')
plt.ylabel('Loss - acc')
plt.legend()
plt.show()